In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f, encoding='latin1')
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)
    

image_size = 28
num_labels = 10
num_channels = 1 # grayscale

'''
Reformat into a TensorFlow-friendly shape:
    convolutions need the image data formatted as a cube (width by height by #channels)
    labels as float 1-hot encodings.
'''
def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('After reformat......')
print('Training set', train_dataset.shape, train_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 28, 28) (200000,)
Test set (10000, 28, 28) (10000,)
After reformat......
Training set (200000, 28, 28, 1) (200000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [2]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

x = tf.placeholder(tf.float32, [None, image_size, image_size, num_channels])
y = tf.placeholder(tf.float32, [None, num_labels])

# Parameters
layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
layer1_biases = tf.Variable(tf.zeros([depth]))
layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

# Define Model
def model(input_image):
    conv1 = tf.nn.conv2d(input_image, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + layer2_biases)
    
    # Flatten previous layer result
    shape = hidden2.get_shape().as_list()
    reshape = tf.reshape(hidden2, [-1, shape[1] * shape[2] * shape[3]])

    # Feed into fully connected layer
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases

# Training computation.
logits = model(x)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))

# Optimizer
optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

num_steps = 2001

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = { x: batch_data, y: batch_labels}
        _, l, train_accuracy_ = sess.run([optimizer, loss, accuracy], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step {}'.format(step, l))
            print('Minibatch accuracy: {}'.format(train_accuracy_))
    
    '''
    Testing Part
    '''        
    print('Testing......')
    feed_dict = { x: test_dataset, y: test_labels}
    test_accuracy_ = sess.run(accuracy, feed_dict=feed_dict)
    print('Test accuracy: {}'.format(test_accuracy_))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Initialized
Minibatch loss at step 0
Minibatch accuracy: 0.0625
Minibatch loss at step 100
Minibatch accuracy: 0.5
Minibatch loss at step 200
Minibatch accuracy: 0.9375
Minibatch loss at step 300
Minibatch accuracy: 0.875
Minibatch loss at step 400
Minibatch accuracy: 1.0
Minibatch loss at step 500
Minibatch accuracy: 0.8125
Minibatch loss at step 600
Minibatch accuracy: 0.8125
Minibatch loss at step 700
Minibatch accuracy: 0.75
Minibatch loss at step 800
Minibatch accuracy: 0.875
Minibatch loss at step 900
Minibatch accuracy: 0.875
Minibatch loss at step 1000
Minibatch accuracy: 0.875
Minibatch loss at step 1100
Minibatch accuracy: 0.75
Minibatch loss at step 1200
Minibatch accuracy: 0.6875
Minibatch loss at step 1300
Minibatch accuracy: 0.625
Minibatch loss at step 1400
Minibatch accuracy: 0.6875